In [ ]:
import csv
import os
import pandas as pd
import yaml

In [ ]:
# Setup Reference Variables
path_to_raw_csv = 'data/raw_banking_export/'
source_file = sorted(os.listdir(os.path.abspath(path_to_raw_csv)), reverse=True)[0]
print(source_file)
full_source_path = f'{path_to_raw_csv}/{source_file}'
print(full_source_path)
working_filepath = 'data/working/working.csv'

config_path = 'config/config.yaml'
start_date = '2024-02-29'

In [ ]:
# Read Config
config = {}
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

In [ ]:
# Reformat the exported transactions to only have meaningful data
with open(full_source_path, 'r') as f:
    csv_reader = csv.reader(f)
    with open(working_filepath, 'w') as fw:
        csv_writer = csv.writer(fw)
        writing = False
        for r in csv_reader:
            if 'Account' in r:
                writing = True

            if writing:
                csv_writer.writerow(r[1:])


In [ ]:
# Read in the working CSV that was just created
df = pd.read_csv(working_filepath, na_values=['0', 'NULL', 'NA'])
# Convert Datatypes
df['Amount'] = df['Amount'].str.replace(',', '')
df['Amount'] = pd.to_numeric(df['Amount'])
df['Date'] = pd.to_datetime(df['Date'], format='mixed')

# df.head()

In [ ]:
# Get only dates after 2/29/24
df = df[df['Date'] >= pd.to_datetime(start_date)]

# df.sort_values(by=['Date'], ascending=False).head(20)

In [ ]:
# Get only transactions from relevant accounts
account_list = []
for accounts in config['accounts']:
    account_list.append(accounts['account'])
df = df[df['Account'].isin(account_list)]

# df.head()

In [ ]:
# Get only cleared transactions
df = df[df['Clr'] == 'R']

# df.head()

In [ ]:
# Clean data
df['Memo/Notes'] = df['Memo/Notes'].fillna('')
df['FITID'] = df['FITID'].fillna('')
df['Payee'] = df['Payee'].str.replace('{', ' - ').str.replace('}', '')
df['Split'] = df['Split'].fillna('')
df['Scheduled'] = df['Scheduled'].fillna('')

# df.head(20)

In [ ]:
# Set the Category to the last item in the split on :
df['Category'] = df['Category'].str.split(':').str[-1]

# df.head()

In [ ]:
# Create a column to be used in split aggs
df['Memo/Notes'] = df.apply(
    lambda x: 
        f"{x['Category']} {x['Amount']} ({x['Memo/Notes']})"
        if (x['Split'] == 'S' and x['Memo/Notes'] != '')
        else (
            f"{x['Category']} {x['Amount']}" 
            if x['Split'] == 'S'
            else x['Memo/Notes']
        ),
    axis='columns'
)

# df.head(20)

In [ ]:
# Group by FITID to resolve Splits
df = df.groupby(['Date', 'Account', 'Payee', 'FITID']) \
    .agg({
        "Category": "|".join,
        "Amount": "sum",
        "Memo/Notes": "|".join
        }) \
    .reset_index() \
    .round(2)

# df.sort_values(by=['Date'], ascending=False).head(20)

In [ ]:
# Set category for income payees
df['Category'] = df.apply(
    lambda x: 'Income' if x['Payee'] in config['income_payees'] else x['Category'],
    axis='columns'
)

# df.head(20)

In [ ]:
# Clean up Transfers
df['Payee'] = df.apply(
    lambda x: x['Category'].replace('[', '').replace(']', '') if ('[' in x['Category'] and ']' in x['Category']) else x['Payee'],
    axis='columns'
)

df['Category'] = df.apply(
    lambda x: 'Transfer' if ('[' in x['Category'] and ']' in x['Category']) else x['Category'],
    axis='columns'
)

In [ ]:
# Rename headers
df = df.rename(columns={
    'Date': 'date',
    'Account': 'account',
    'Payee': 'payee',
    'Memo/Notes': 'notes',
    'Category': 'category',
    'Amount': 'amount',
    'FITID': 'fitid'
    })

# df.head(50)

In [ ]:
# Write the new full dataset to CSV
print(f"{os.path.abspath('data/processed/')}/full.csv")
df.to_csv(f"{os.path.abspath('data/processed/')}/full.csv", index=False)

# Write individual files out
for account in df['account'].unique():
    print(account)
    new_df = df[df['account'] == account]
    final_df = new_df[['date', 'payee', 'notes', 'category', 'amount']]
    final_df.to_csv(f"{os.path.abspath('data/processed/')}/{account}.csv".replace(' ', '').replace("'", ''), index=False)
